# Databricks Native ST geospatial functions

Databricks SQL includes a large number of [ST geospatial functions](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-st-geospatial-functions) for large-scale, native processing of geodata. Just a few examples:

## GEOGRAPHY operations

You can use `st_area`, `st_length` and `st_perimeter` directly on GEOGRAPHY (lon/lat) columns to get results in meters:

In [0]:
%sql
-- Areas of countries. Note that a country might not be listed if:
-- - they don't have generally accepted borders, or
-- - they cross the date line, leading to unsupported coordinates for `st_geogfromwkb` (hence the use of `try_to_geography`)
select
  country, names:primary name,
  st_area(try_to_geography(geometry)) / 1e6 area_km2
from
  carto_overture_maps_divisions.carto.division_area
where
  subtype = 'country'
  and class = 'land'
order by
  area_km2 
-- Returns:
-- country	name	area_km2
-- CA	"""Canada"""	9966895.526025355
-- US	"""United States"""	9476994.623136332
-- CN	"""中国"""	9390439.241133066
-- BR	"""Brasil"""	8507809.984099092
-- IN	"""India"""	3149764.8857977577
-- ...

If you store (lon, lat) point data as GEOGRAPHY, you can calculate distance in meters by converting to `GEOMETRY(4326)` first:

In [0]:
%sql
with cities as (
  select
    names:primary::string as name,
    st_geogfromwkb(geometry) geog
  from
    carto_overture_maps_divisions.carto.division
  where
    subtype = 'locality'
    and class = 'city'
    -- just to speed up the lookup
    and `__carto_xmax` between - 1.41 and 2.99
    and `__carto_ymax` between 48.57 and 52.21
),
london as (
  select
    *
  from
    cities
  where
    name = "London"
),
paris as (
  select
    *
  from
    cities
  where
    name = "Paris"
)
select
  st_distancespheroid(london.geog::geometry(4326), paris.geog::geometry(4326)) / 1e3 dist_km
from
  london,
  paris
--
-- Returns:
--
-- dist_km
-- 344.08532856606695